# 1.图片存储为image

In [5]:
import fitz  # PyMuPDF
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
base_dir = '/data/coding/patent_qa/train/'
pdf_file_list  = [x for x in os.listdir(base_dir+'/documents/') if 'pdf' in x]

for file_name in tqdm(pdf_file_list):
    pdf_document = fitz.open(base_dir+'/documents/'+file_name)
    os.makedirs(base_dir+'/pdf_img/'+file_name.split('.')[0],exist_ok=True)
    # 获取第一页
    for i in range(pdf_document.page_count):
        page = pdf_document.load_page(i)  # 注意：页码从0开始
        # 将页面转换为图像
        pix = page.get_pixmap(dpi=600) # 这些文档600的dpi够了
        pix.save(base_dir+'/pdf_img/'+file_name.split('.')[0]+'/'+str(i+1)+'.jpg')

100%|██████████| 51/51 [13:32<00:00, 15.93s/it]


In [6]:
# 2.OCR

In [1]:
from swift.llm import InferEngine, InferRequest, PtEngine, RequestConfig, load_dataset
from swift.plugin import InferStats
from swift.llm import VllmEngine
import os
os.environ['VIDEO_MAX_PIXELS'] = '50176'
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["MAX_PIXELS"] = "1229312"
os.environ['FPS_MAX_FRAMES'] = "2"
model_path = "/data/coding/llm_model/Qwen/Qwen2___5-VL-7B-Instruct/"
# 多卡设置tensor_parallel_size为卡数
engine = VllmEngine(model_path,model_type='qwen2_5_vl',gpu_memory_utilization=0.9,limit_mm_per_prompt={"image": 1},tensor_parallel_size=2)
def infer_batch(engine, infer_requests):
    request_config = RequestConfig(max_tokens=10240, temperature=0)
    metric = InferStats()
    resp_list = engine.infer(infer_requests, request_config)
    response = resp_list[0].choices[0].message.content
    return response

/data/miniconda/envs/swift/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[INFO:swift] Successfully registered `/data/miniconda/envs/swift/lib/python3.10/site-packages/swift/llm/dataset/data/dataset_info.json`.


INFO 05-23 19:34:43 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-23 19:34:43 [__init__.py:239] Automatically detected platform cuda.


2025-05-23 19:34:45,486	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
[INFO:swift] Loading the model using model_dir: /data/coding/llm_model/Qwen/Qwen2___5-VL-7B-Instruct/
[INFO:swift] Setting torch_dtype: torch.bfloat16
[WARNING:swift] Please install the package: `pip install "decord" -U`.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
[INFO:swift] Setting image_factor: 28. You can adjust this hyperparameter through the environment variable: `IMAGE_FACTOR`.
[INFO:swift] Setting min_pixels: 3136. You can adjust this hyperparameter through the environment variable: `MIN_PIXELS`.
[INFO:swift] Using environment variabl

INFO 05-23 19:34:54 [config.py:717] This model supports multiple tasks: {'score', 'reward', 'generate', 'classify', 'embed'}. Defaulting to 'generate'.
WARNING 05-23 19:34:54 [arg_utils.py:1536] The model has a long context length (128000). This may causeOOM during the initial memory profiling phase, or result in low performance due to small KV cache size. Consider setting --max-model-len to a smaller value.
INFO 05-23 19:34:54 [config.py:1770] Defaulting to use mp for distributed inference
INFO 05-23 19:34:54 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='/data/coding/llm_model/Qwen/Qwen2___5-VL-7B-Instruct', speculative_config=None, tokenizer='/data/coding/llm_model/Qwen/Qwen2___5-VL-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=128000, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_p

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:01,  3.27it/s]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:01<00:02,  1.10it/s]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:03<00:02,  1.16s/it]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:04<00:01,  1.30s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:06<00:00,  1.36s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:06<00:00,  1.21s/it]


...

Capturing CUDA graph shapes:  86%|████████▌ | 30/35 [00:21<00:03,  1.41it/s]

(VllmWorkerProcess pid=3347) INFO 05-23 19:37:05 [custom_all_reduce.py:195] Registering 1995 cuda graph addresses


Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:26<00:00,  1.33it/s]

INFO 05-23 19:37:10 [custom_all_reduce.py:195] Registering 1995 cuda graph addresses
(VllmWorkerProcess pid=3347) INFO 05-23 19:37:10 [model_runner.py:1592] Graph capturing finished in 26 secs, took 0.25 GiB
INFO 05-23 19:37:10 [model_runner.py:1592] Graph capturing finished in 26 secs, took 0.25 GiB
INFO 05-23 19:37:10 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 106.90 seconds


WARNING 05-23 19:37:11 [sampling_params.py:347] temperature 1e-06 is less than 0.01, which may cause numerical errors nan or inf in tensors. We have maxed it out to 0.01.


In [7]:
def qwen_ocr_cn(img_path):
    prompt="你是一个OCR专家，请提取以下图片中的所有文字内容，并原样返回。"
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": img_path,
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]

    data = dict()
    data['messages'] = messages
    infer_requests = [InferRequest(**data)]
    response = infer_batch(engine, infer_requests)
    return response

def qwen_ocr_en(img_path):
    prompt="You are an OCR expert. Please extract all text from the following images, which have been converted from PDFs, and return the exact text content found in the attached image."
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": img_path,
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]

    data = dict()
    data['messages'] = messages
    infer_requests = [InferRequest(**data)]
    response = infer_batch(engine, infer_requests)
    return response

In [9]:
# 进行ocr
import fitz  # PyMuPDF
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
base_dir = '/data/coding/patent_qa/train/'
pdf_file_list = [x for x in os.listdir(base_dir+'/pdf_img/')]
for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_img/'+file_name) if 'jpg' in x] # pdf文件存储的jpg
    os.makedirs(base_dir+'/pdf_ocr/'+file_name,exist_ok=True)
    for k in tqdm(range(len(file_list))):
        img_path = base_dir+'/pdf_img/'+file_name + '/' + file_list[k]
        page_num = int(file_list[k].split('.')[0])
        # ocr
        response = qwen_ocr_cn(img_path)
        # 保存ocr结果为txt
        with open(base_dir+f'/pdf_ocr/{file_name}/{page_num}.txt', 'w', encoding='utf-8') as text_file:
            text_file.write(response)
    print(file_name,'  ok')

100%|██████████| 19/19 [01:35<00:00,  5.04s/it]


CN100364694C   ok


100%|██████████| 10/10 [01:18<00:00,  7.83s/it]


CN101134389B   ok


100%|██████████| 10/10 [01:04<00:00,  6.44s/it]


...

CN217268780U   ok


100%|██████████| 5/5 [00:32<00:00,  6.52s/it]


CN217479855U   ok


100%|██████████| 7/7 [00:43<00:00,  6.26s/it]

CN218108941U   ok


# 2. OCR的结果存入向量库

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1'
os.environ["MAX_PIXELS"] = '1229312' # 1003520
from gme_inference import GmeQwen2VL
gme = GmeQwen2VL('/home/octopus/data/dxw/pretrain_model/iic/gme-Qwen2-VL-7B-Instruct',max_image_tokens=1280)

In [ ]:
import os
import pandas as pd
import numpy as np
import tqdm
from warnings import filterwarnings
# 过滤掉一些警告
filterwarnings("ignore")
base_dir = '/data/coding/patent_qa/train/'
pdf_file_list = [x for x in os.listdir(base_dir+'/pdf_img/')]
files_total_cnt = 0
for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_ocr/'+file_name) if 'txt' in x]
    files_total_cnt +=len(file_list)

In [ ]:
# 文件有多个，每个都需要存映射关系
ocr_page_num_list = []
ocr_name_list = []
ocr_vectors = np.empty((files_total_cnt, 3584)) # 向量维度是3584维
idx = 0
for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_ocr/'+file_name) if 'txt' in x] # pdf文件存储的jpg
    for k in range(len(file_list)):  
        text_path = base_dir+'/pdf_ocr/'+file_name + '/' + file_list[k]
        with open(text_path, 'r', encoding='utf-8') as f:
            text_content = f.read()
        e_text = gme.get_text_embeddings(texts=[text_content])
        ocr_vectors[idx] = e_text[0].to('cpu').numpy()
        page_num = int(file_list[k].split('.')[0])
        ocr_page_num_list.append(page_num)
        ocr_name_list.append(file_name)
        idx+=1
# 映射关系存储到pandas里面比较方便
ocr_page_num_mapping = pd.DataFrame({'index': range(len(ocr_page_num_list)), 'page_num': ocr_page_num_list, 'file_name': ocr_name_list})
# 将向量和页码映射关系存储到文件
np.save('ocr_vectors.npy', ocr_vectors)
ocr_page_num_mapping.to_csv('ocr_page_num_mapping.csv', index=False) # 存储映射关系


# 3. 图片的结果存入向量库

In [ ]:
import os
import pandas as pd
import numpy as np
import tqdm
from warnings import filterwarnings
# 过滤掉一些警告
filterwarnings("ignore")
base_dir = '/data/coding/patent_qa/train/'
pdf_file_list = [x for x in os.listdir(base_dir+'/pdf_img/')]
files_total_cnt = 0
for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_img/'+file_name) if 'jpg' in x]
    files_total_cnt +=len(file_list)

In [ ]:
# 文件有多个，每个都需要存映射关系
img_page_num_list = []
img_name_list = []
img_vectors = np.empty((files_total_cnt, 3584)) # 向量维度是3584维
idx = 0
for pdf_file in pdf_file_list:
    file_name = pdf_file.split('.')[0]
    file_list = [x for x in os.listdir(base_dir+'/pdf_img/'+file_name) if 'jpg' in x] # pdf文件存储的jpg
    for k in range(len(file_list)):  
        image_path = base_dir+'/pdf_img/'+file_name + '/' + file_list[k]
        e_text = gme.get_image_embeddings(images=[image_path])
        img_vectors[idx] = e_text[0].to('cpu').numpy()
        page_num = int(file_list[k].split('.')[0])
        img_page_num_list.append(page_num)
        img_name_list.append(file_name)
        idx+=1
# 映射关系存储到pandas里面比较方便
img_page_num_mapping = pd.DataFrame({'index': range(len(img_page_num_list)), 'page_num': img_page_num_list, 'file_name': img_name_list})
# 将向量和页码映射关系存储到文件
np.save('pdf_img_vectors.npy', img_vectors)
img_page_num_mapping.to_csv('pdf_img_page_num_mapping.csv', index=False) # 存储映射关系

# 4. 读取问题生成问题的向量

In [ ]:
# 问题的vector进行保存
question_vectors = np.empty((len(df_question), 3584))
for i in range(len(df_question)):
    question = df_question.loc[i,'question']
    document_name = df_question.loc[i,'document']
    options = df_question.loc[i,'options']
    true_answer = df_question.loc[i,'answer']
    full_question = question + ' '.join(options)
    query_vec = gme.get_text_embeddings(texts=[full_question])
    question_vectors[i] = query_vec[0].to('cpu').numpy()
# 保存问题的向量
np.save('all_question_vectors.npy', question_vectors)

In [13]:
df_question = pd.read_json("/data/coding/patent_qa/train/questions.jsonl",lines=True)
df_question.head()

,question,document,options,answer,group,id
0,根据专利文本，以下哪个是该货物靠边规整处理机构的主要功能？,CN213444549U.pdf,"[A. 实现对货物的加热处理。, B. 实现对货物的靠边规整处理。, C. 实现对货物的分拣...",B,1,1
1,根据专利文本，关于转辊倾斜角度的描述，以下哪项是正确的？,CN213444549U.pdf,"[A. 转辊的倾斜角度为15-20°。, B. 转辊的倾斜角度为5-10°。, C. 转辊的...",B,1,2
2,在文件中第5页提供的图片中，编号为4的部件是什么？,CN213444549U.pdf,"[A. 转轴, B. 侧板, C. 联动皮带, D. 支架]",C,2,3
3,在文件中第5页提供的图片中，编号为5的部件是什么？,CN213444549U.pdf,"[A. 转辊, B. 支架, C. 联动皮带, D. 侧板]",B,2,4
4,在文件中第5页的示意图中，如果货物靠着侧板1移动，那么最可能的原因是？,CN213444549U.pdf,"[A. 转辊2是水平设置的, B. 联动皮带4松动, C. 货物移动到了侧边并且不能再移动,...",C,3,5


In [ ]:
def get_text_answer(document_name,question,options,i):
    ...

In [ ]:
for i in range(len(df_question)):
    question = df_question.loc[i,'question']
    document_name = df_question.loc[i,'document']
    options = df_question.loc[i,'options']
    true_answer = df_question.loc[i,'answer']
    full_question = question + ' '.join(options)
    if "第" in question and "页" in question and "图": # 问题含有图片
        pic_page_num = re.findall(r"第(\d+)页", question)[0]
        pic_page_num = int(pic_page_num)
        text_answer = get_text_answer_pic_question(document_name,pic_page_num,question,options,i) # 使用含有文本来回答
        image_answer = get_img_answer_pic_question(document_name,pic_page_num,question,options,i) # 仅使用目标图像来回答
    else:
        text_answer = get_text_answer(document_name,question,options,i) # 使用文本来回答
        image_answer = get_img_answer(document_name,question,options,i) # 使用图像来回答
    

In [ ]:
# 

In [15]:
for i in range(len(df_question)):
    question = df_question.loc[i,'question']
    document_name = df_question.loc[i,'document']
    options = df_question.loc[i,'options']
    true_answer = df_question.loc[i,'answer']
    full_question = question + ' '.join(options)
    if "第" in question and "页" in question and "图": # 问题含有图片
        pic_page_num = re.findall(r"第(\d+)页", question)[0]
        pic_page_num = int(pic_page_num)
        text_answer = get_text_answer_pic_question(document_name,pic_page_num,full_question) # 使用含有文本来回答
        image_answer = get_img_answer_pic_question(document_name,pic_page_num,full_question) # 仅使用目标图像来回答
    else:
        text_answer = get_text_answer(document_name,full_question) # 使用文本来回答
        image_answer = get_img_answer(document_name,full_question) # 使用图像来回答
    